# TODO:
* Collect image-concept data
* Separate blooms per concept
* Run sample queries over the blooms
* Compare to: bitmap only, bloom+bitmap, bloom various configs

In [ ]:
import numpy as np
from pyroaring import BitMap

def bitmap_to_pyroaring(bitmap):
    roarings = []
    for col in bitmap.shape[1]:
        roarings.append(BitMap(bitmap[:, col]))
    return roarings

mem_keys = ['n_bytes_array_containers', 'n_bytes_run_containers', 'n_bytes_bitset_containers']
stats = BitMap(np.array([1, 2, 3])).get_statistics()
for key in mem_keys:
    print(key, stats[key])

n_bytes_array_containers 0
n_bytes_run_containers 6
n_bytes_bitset_containers 0


In [ ]:
from pympler import asizeof
from scipy import sparse
import rbloom
import numpy as np
from pyroaring import BitMap
import time
from itertools import combinations
import pandas as pd
from typing import List, Tuple

def generate_sparse_bitmap(size: int, sparsity: float, mean_sparsity: bool=True) -> np.ndarray:
    """Generate a sparse bitmap with given size and sparsity level."""
    num_ones = int(size * sparsity)
    if mean_sparsity:
        r = np.random.randint(1, int(sparsity*1000)) / 1000
        num_ones = int(size * r)
    indices = np.random.choice(size, num_ones, replace=False)
    bitmap = np.zeros(size, dtype=bool)
    bitmap[indices] = True
    return bitmap

def generate_test_case(size: int, sparsity: float, num_bitmaps: int) -> Tuple[np.ndarray, List[BitMap]]:
    """Generate test case with numpy bitmaps and equivalent PyRoaring bitmaps."""
    # Generate numpy bitmaps
    np_bitmaps = np.array([generate_sparse_bitmap(size, sparsity) for _ in range(num_bitmaps)])
    # print(np.sum(np_bitmaps, axis=1), 'sparse', sparsity, 'n', num_bitmaps)
    # np_bitmaps = np.asfortranarray(np_bitmaps)
    
    # Convert to PyRoaring
    roaring_bitmaps = [BitMap(np.where(bitmap)[0]) for bitmap in np_bitmaps]

    # Convert to blooms
    # blooms = [rbloom.Bloom(bitmap, 1/10_000) for bitmap in np_bitmaps]
    choose_densest = True
    bloom_size = size
    expansion = 1.01
    if choose_densest:
        bloom_size = int(np.max(np.sum(np_bitmaps, axis=1)) * expansion)

    # blooms = [rbloom.Bloom(size, 1/1_000) for _ in range(num_bitmaps)]
    blooms = [rbloom.Bloom(bloom_size, 1/1_000) for _ in range(num_bitmaps)]
    for i, bitmap in enumerate(np_bitmaps):
        blooms[i].update(np.where(bitmap)[0])

    
    return np_bitmaps, roaring_bitmaps, blooms

def benchmark_numpy_intersection(bitmaps: np.ndarray) -> Tuple[float, int]:
    """Benchmark numpy bitmap intersection."""
    # bitmaps = np.asfortranarray(bitmaps)
    start_time = time.time()
    result = np.all(bitmaps, axis=0)
    result_values = np.where(result)[0]
    count = len(result_values)
    duration = time.time() - start_time
    return duration, count, bitmaps.nbytes / (1024 * 1024)

def benchmark_pyroaring_intersection(bitmaps: List[BitMap], scale_len) -> Tuple[float, int]:
    """Benchmark PyRoaring bitmap intersection."""
    start_time = time.time()
    result = bitmaps[0].copy()
    for bitmap in bitmaps[1:]:
        result &= bitmap

    count = 0
    # for i in range(scale_len):
        # if i in result:
            # count += 1
    # values = np.where(result)[0]
    # count = len(values)
    result_values = list(result)
    count = len(result_values)

    # count = len(result)
    duration = time.time() - start_time
    nbytes = 0
    mem_keys = ['n_bytes_array_containers', 'n_bytes_run_containers', 'n_bytes_bitset_containers']
    for bitmap in bitmaps:
        stats = bitmap.get_statistics()
        for key in mem_keys:
            nbytes += stats[key]

    # print(nbytes / (1024 * 1024), '|', sum([asizeof.asizeof(bitmap) / (1024 * 1024) for bitmap in bitmaps])) 
    return duration, count, nbytes / (1024 * 1024)

def benchmark_blooms_intersection(bitmaps, scan_len) -> Tuple[float, int]:
    """Benchmark BloomFilter intersection."""
    start_time = time.time()
    result = bitmaps[0].copy()
    for bloom in bitmaps[1:]:
        result &= bloom

    count = 0
    for i in range(scan_len):
        if i in result:
            count += 1
    duration = time.time() - start_time
    # count = len(result)

    nbytes = 0
    for bloom in bitmaps:
        nbytes += bloom.size_in_bits / 8

    return duration, count, nbytes / (1024 * 1024)


def run_benchmarks():
    """Run comprehensive benchmarks and return results as a DataFrame."""
    sizes = [100_000, 1_000_000]
    sparsities = [0.01, 0.1, 0.2]
    num_bitmaps_list = [1, 2, 4, 8]
    num_trials = 5
    
    results = []
    
    for size in sizes:
        for sparsity in sparsities:
            for num_bitmaps in num_bitmaps_list:
                # print(f"Running benchmark: size={size}, sparsity={sparsity}, num_bitmaps={num_bitmaps}")
                
                numpy_times = []
                roaring_times = []
                blooms_times = []
                intersection_sizes = []

                numpy_mem_storage = []
                roaring_mem_storage = []
                blooms_mem_storage = []
                sparse_storage = []
                for trial in range(num_trials):
                    # Generate test case
                    np_bitmaps, roaring_bitmaps, blooms = generate_test_case(size, sparsity, num_bitmaps)
                    
                    # Run benchmarks
                    numpy_time, numpy_count, numpy_nbytes = benchmark_numpy_intersection(np_bitmaps)
                    roaring_time, roaring_count, roaring_nbytes = benchmark_pyroaring_intersection(roaring_bitmaps, size)
                    blooms_time, blooms_count, blooms_nbytes = benchmark_blooms_intersection(blooms, size)
                    
                    # Verify results match
                    assert numpy_count == roaring_count, "Intersection results don't match!"
                    # print(f"{blooms_count}|{numpy_count}")
                    if blooms_count != numpy_count:
                        # print(f"Bloom count {blooms_count} does not match numpy count {numpy_count}")
                        n_fp = blooms_count - numpy_count
                        # print(f"Bloom count {blooms_count} does not match numpy count {numpy_count}, {n_fp} false positives for {n_fp/blooms_count} false positive rate")
                    
                    numpy_times.append(numpy_time)
                    roaring_times.append(roaring_time)
                    blooms_times.append(blooms_time)
                    intersection_sizes.append(numpy_count)

                    sparse_mat_bytes = asizeof.asizeof(sparse.csr_matrix(np_bitmaps))

                    sparse_storage.append(sparse_mat_bytes / (1024 * 1024))
                    numpy_mem_storage.append(numpy_nbytes)
                    roaring_mem_storage.append(roaring_nbytes)
                    blooms_mem_storage.append(blooms_nbytes)

                
                # Calculate averages
                avg_numpy_time = np.mean(numpy_times)
                avg_roaring_time = np.mean(roaring_times)
                avg_blooms_time = np.mean(blooms_times)
                avg_intersection_size = np.mean(intersection_sizes)
                
                results.append({
                    'size': size,
                    'sparsity': sparsity,
                    'num_bitmaps': num_bitmaps,
                    'numpy_time': avg_numpy_time,
                    'roaring_time': avg_roaring_time,
                    'blooms_time': avg_blooms_time,
                    'intersection_size': avg_intersection_size,
                    'speedup': avg_numpy_time / avg_roaring_time,
                    'blooms_speedup': avg_numpy_time / avg_blooms_time,
                    'numpy_mem_storage': np.mean(numpy_mem_storage),
                    'sparse_mem_storage': np.mean(sparse_storage),
                    'roaring_mem_storage': np.mean(roaring_mem_storage),
                    'blooms_mem_storage': np.mean(blooms_mem_storage)

                })
    
    return pd.DataFrame(results)

# Set random seed for reproducibility
np.random.seed(42)

# Run benchmarks
results_df = run_benchmarks()

# Print results
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '{:.6f}'.format(x))
print("\nBenchmark Results:")
display(results_df)

# Save results to CSV
# results_df.to_csv('bitmap_benchmark_results.csv', index=False)

# Print summary statistics
print("\nSummary Statistics:")
print(f"Average speedup: {results_df['speedup'].mean():.2f}x")
print(f"Maximum speedup: {results_df['speedup'].max():.2f}x")
print(f"Minimum speedup: {results_df['speedup'].min():.2f}x")


Benchmark Results:


,size,sparsity,num_bitmaps,numpy_time,roaring_time,blooms_time,intersection_size,speedup,blooms_speedup,numpy_mem_storage,sparse_mem_storage,roaring_mem_storage,blooms_mem_storage
0,100000,0.010000,1,0.000062,0.000013,0.007997,380.000000,4.769231,0.007764,0.095367,0.004765,0.000725,0.000658
1,100000,0.010000,2,0.000024,0.000006,0.006613,2.400000,3.885496,0.003670,0.190735,0.008777,0.001525,0.001867
2,100000,0.010000,4,0.000023,0.000007,0.006352,0.000000,3.156863,0.003626,0.381470,0.018513,0.003471,0.005119
3,100000,0.010000,8,0.000035,0.000007,0.006386,0.000000,4.793548,0.005548,0.762939,0.035313,0.006827,0.010786
4,100000,0.100000,1,0.000096,0.000133,0.008260,7140.000000,0.722262,0.011650,0.095367,0.069231,0.011040,0.012360
5,100000,0.100000,2,0.000028,0.000017,0.006784,190.000000,1.660920,0.004063,0.190735,0.088501,0.015793,0.022365
6,100000,0.100000,4,0.000025,0.000014,0.006497,0.200000,1.717105,0.003831,0.381470,0.196471,0.036033,0.052208
7,100000,0.100000,8,0.000039,0.000019,0.006472,0.000000,2.063291,0.006005,0.762939,0.413158,0.072479,0.124918
8,100000,0.200000,1,0.000065,0.000164,0.008415,9079.800000,0.397552,0.007729,0.095367,0.087733,0.013253,0.015718
9,100000,0.200000,2,0.000043,0.000040,0.006963,1206.400000,1.094089,0.006211,0.190735,0.220493,0.028166,0.050755



Summary Statistics:
Average speedup: 2.90x
Maximum speedup: 10.08x
Minimum speedup: 0.40x


In [54]:
def generate_sparse_bitmap(size: int, sparsity: float) -> np.ndarray:
    """Generate a sparse bitmap with given size and sparsity level."""
    num_ones = int(size * sparsity)
    indices = np.random.choice(size, num_ones, replace=False)
    bitmap = np.zeros(size, dtype=bool)
    bitmap[indices] = True
    return bitmap

b = np.array([generate_sparse_bitmap(10, 0.5) for _ in range(10)])
b.sum(axis=0)


size = 1000
sparsity = 0.1
num_bitmaps = 10
c = np.array([generate_sparse_bitmap(size, sparsity) for _ in range(num_bitmaps)])
print(c.sum(axis=1))


[100 100 100 100 100 100 100 100 100 100]


In [11]:
results_df['blooms_mem_storage']

,size,sparsity,num_bitmaps,numpy_time,roaring_time,blooms_time,intersection_size,speedup,blooms_speedup,numpy_mem_storage,sparse_mem_storage,roaring_mem_storage,blooms_mem_storage
0,100000,0.010000,1,0.000088,0.000013,0.008027,380.000000,6.494700,0.010919,0.095367,0.004765,0.000725,0.000438
1,100000,0.010000,2,0.000023,0.000007,0.006764,2.400000,3.408451,0.003412,0.190735,0.008777,0.001525,0.001245
2,100000,0.010000,4,0.000022,0.000007,0.006516,0.000000,2.929936,0.003366,0.381470,0.018513,0.003471,0.003413
3,100000,0.010000,8,0.000034,0.000009,0.006605,0.000000,3.977654,0.005140,0.762939,0.035313,0.006827,0.007191
4,100000,0.100000,1,0.000096,0.000147,0.008322,7140.000000,0.650680,0.011506,0.095367,0.069231,0.011040,0.008240
5,100000,0.100000,2,0.000029,0.000017,0.006718,190.000000,1.709402,0.004259,0.190735,0.088501,0.015793,0.014910
6,100000,0.100000,4,0.000025,0.000015,0.006483,0.200000,1.693548,0.003861,0.381470,0.196471,0.036033,0.034805
7,100000,0.100000,8,0.000038,0.000018,0.006485,0.000000,2.062338,0.005838,0.762939,0.413158,0.072479,0.083276
8,100000,0.200000,1,0.000065,0.000175,0.008365,9079.800000,0.372201,0.007769,0.095367,0.087733,0.013253,0.010479
9,100000,0.200000,2,0.000043,0.000040,0.006983,1206.400000,1.056805,0.006098,0.190735,0.220493,0.028166,0.033837


In [25]:
import numpy as np
from pyroaring import BitMap
import time
from itertools import combinations
import pandas as pd
from typing import List, Tuple
import sys
import psutil
import os

def get_size_mb(obj) -> float:
    """Get size of object in MB."""
    return sys.getsizeof(obj) / (1024 * 1024)

def get_process_memory() -> float:
    """Get current process memory usage in MB."""
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / (1024 * 1024)

def generate_sparse_bitmap(size: int, sparsity: float) -> np.ndarray:
    """Generate a sparse bitmap with given size and sparsity level."""
    num_ones = int(size * sparsity)
    indices = np.random.choice(size, num_ones, replace=False)
    bitmap = np.zeros(size, dtype=bool)
    bitmap[indices] = True
    return bitmap

def generate_test_case(size: int, sparsity: float, num_bitmaps: int) -> Tuple[np.ndarray, List[BitMap]]:
    """Generate test case with numpy bitmaps and equivalent PyRoaring bitmaps."""
    # Generate numpy bitmaps
    np_bitmaps = np.array([generate_sparse_bitmap(size, sparsity) for _ in range(num_bitmaps)])
    
    # Convert to PyRoaring
    roaring_bitmaps = [BitMap(np.where(bitmap)[0]) for bitmap in np_bitmaps]
    
    return np_bitmaps, roaring_bitmaps

def measure_memory_usage(bitmaps) -> Tuple[float, float]:
    """Measure memory usage of bitmap storage and during intersection."""
    if isinstance(bitmaps, np.ndarray):
        storage_mem = bitmaps.nbytes / (1024 * 1024)  # Convert to MB
    else:  # PyRoaring bitmaps
        storage_mem = sum(bitmap.get_statistics().get('bytes_allocated', 0) for bitmap in bitmaps) / (1024 * 1024)
    
    base_mem = get_process_memory()
    return storage_mem, base_mem

def benchmark_numpy_intersection(bitmaps: np.ndarray) -> Tuple[float, int, float]:
    """Benchmark numpy bitmap intersection with memory tracking."""
    start_mem = get_process_memory()
    start_time = time.time()
    result = np.all(bitmaps, axis=0)
    duration = time.time() - start_time
    peak_mem = get_process_memory()
    count = np.sum(result)
    return duration, count, peak_mem - start_mem

def benchmark_pyroaring_intersection(bitmaps: List[BitMap]) -> Tuple[float, int, float]:
    """Benchmark PyRoaring bitmap intersection with memory tracking."""
    start_mem = get_process_memory()
    start_time = time.time()
    result = bitmaps[0].copy()
    for bitmap in bitmaps[1:]:
        result &= bitmap
    duration = time.time() - start_time
    peak_mem = get_process_memory()
    count = len(result)
    return duration, count, peak_mem - start_mem

def run_benchmarks():
    """Run comprehensive benchmarks and return results as a DataFrame."""
    sizes = [1_000, 100_000, 1_000_000]
    sparsities = [0.01, 0.1]
    num_bitmaps_list = [1, 2, 4, 8]
    num_trials = 5
    
    results = []
    
    for size in sizes:
        for sparsity in sparsities:
            for num_bitmaps in num_bitmaps_list:
                print(f"Running benchmark: size={size}, sparsity={sparsity}, num_bitmaps={num_bitmaps}")
                
                numpy_times = []
                roaring_times = []
                numpy_mem_storage = []
                roaring_mem_storage = []
                numpy_mem_peak = []
                roaring_mem_peak = []
                intersection_sizes = []
                
                for trial in range(num_trials):
                    # Generate test case
                    np_bitmaps, roaring_bitmaps = generate_test_case(size, sparsity, num_bitmaps)
                    
                    # Measure memory usage
                    np_storage, np_base = measure_memory_usage(np_bitmaps)
                    roar_storage, roar_base = measure_memory_usage(roaring_bitmaps)
                    
                    numpy_mem_storage.append(np_storage)
                    roaring_mem_storage.append(roar_storage)
                    
                    # Run benchmarks
                    numpy_time, numpy_count, np_peak = benchmark_numpy_intersection(np_bitmaps)
                    roaring_time, roaring_count, roar_peak = benchmark_pyroaring_intersection(roaring_bitmaps)
                    
                    # Verify results match
                    assert numpy_count == roaring_count, "Intersection results don't match!"
                    
                    numpy_times.append(numpy_time)
                    roaring_times.append(roaring_time)
                    numpy_mem_peak.append(np_peak)
                    roaring_mem_peak.append(roar_peak)
                    intersection_sizes.append(numpy_count)
                
                # Calculate averages
                results.append({
                    'size': size,
                    'sparsity': sparsity,
                    'num_bitmaps': num_bitmaps,
                    'numpy_time': np.mean(numpy_times),
                    'roaring_time': np.mean(roaring_times),
                    'numpy_storage_mb': np.mean(numpy_mem_storage),
                    'roaring_storage_mb': np.mean(roaring_mem_storage),
                    'numpy_peak_mb': np.mean(numpy_mem_peak),
                    'roaring_peak_mb': np.mean(roaring_mem_peak),
                    'intersection_size': np.mean(intersection_sizes),
                    'time_speedup': np.mean(numpy_times) / np.mean(roaring_times),
                    'storage_ratio': np.mean(numpy_mem_storage) / np.mean(roaring_mem_storage),
                    'peak_mem_ratio': np.mean(numpy_mem_peak) / np.mean(roaring_mem_peak)
                })
    
    return pd.DataFrame(results)

# Set random seed for reproducibility
np.random.seed(42)

# Run benchmarks
results_df = run_benchmarks()

# Print results
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '{:.6f}'.format(x))

# Sort and display results
sorted_results = results_df.sort_values(['size', 'sparsity', 'num_bitmaps'])
print("\nBenchmark Results:")
print(sorted_results)

# Print summary statistics
print("\nSummary Statistics:")
print("\nTime Performance:")
print(f"Average speedup: {results_df['time_speedup'].mean():.2f}x")
print(f"Maximum speedup: {results_df['time_speedup'].max():.2f}x")
print(f"Minimum speedup: {results_df['time_speedup'].min():.2f}x")

print("\nMemory Usage:")
print(f"Average storage ratio (NumPy/Roaring): {results_df['storage_ratio'].mean():.2f}x")
print(f"Average peak memory ratio (NumPy/Roaring): {results_df['peak_mem_ratio'].mean():.2f}x")

# Group by sparsity and size
print("\nPerformance by Sparsity:")
sparsity_summary = results_df.groupby('sparsity').agg({
    'time_speedup': ['mean', 'min', 'max'],
    'storage_ratio': ['mean', 'min', 'max'],
    'peak_mem_ratio': ['mean', 'min', 'max']
})
print(sparsity_summary)

# Save results to CSV
results_df.to_csv('bitmap_benchmark_results.csv', index=False)

Running benchmark: size=1000, sparsity=0.01, num_bitmaps=1
Running benchmark: size=1000, sparsity=0.01, num_bitmaps=2
Running benchmark: size=1000, sparsity=0.01, num_bitmaps=4
Running benchmark: size=1000, sparsity=0.01, num_bitmaps=8
Running benchmark: size=1000, sparsity=0.1, num_bitmaps=1
Running benchmark: size=1000, sparsity=0.1, num_bitmaps=2
Running benchmark: size=1000, sparsity=0.1, num_bitmaps=4
Running benchmark: size=1000, sparsity=0.1, num_bitmaps=8
Running benchmark: size=100000, sparsity=0.01, num_bitmaps=1
Running benchmark: size=100000, sparsity=0.01, num_bitmaps=2
Running benchmark: size=100000, sparsity=0.01, num_bitmaps=4
Running benchmark: size=100000, sparsity=0.01, num_bitmaps=8
Running benchmark: size=100000, sparsity=0.1, num_bitmaps=1
Running benchmark: size=100000, sparsity=0.1, num_bitmaps=2
Running benchmark: size=100000, sparsity=0.1, num_bitmaps=4
Running benchmark: size=100000, sparsity=0.1, num_bitmaps=8


/tmp/ipykernel_2694995/9464567.py:129: RuntimeWarning: divide by zero encountered in scalar divide
  'storage_ratio': np.mean(numpy_mem_storage) / np.mean(roaring_mem_storage),
/tmp/ipykernel_2694995/9464567.py:130: RuntimeWarning: invalid value encountered in scalar divide
  'peak_mem_ratio': np.mean(numpy_mem_peak) / np.mean(roaring_mem_peak)


Running benchmark: size=1000000, sparsity=0.01, num_bitmaps=1
Running benchmark: size=1000000, sparsity=0.01, num_bitmaps=2
Running benchmark: size=1000000, sparsity=0.01, num_bitmaps=4
Running benchmark: size=1000000, sparsity=0.01, num_bitmaps=8
Running benchmark: size=1000000, sparsity=0.1, num_bitmaps=1
Running benchmark: size=1000000, sparsity=0.1, num_bitmaps=2
Running benchmark: size=1000000, sparsity=0.1, num_bitmaps=4
Running benchmark: size=1000000, sparsity=0.1, num_bitmaps=8

Benchmark Results:
       size  sparsity  num_bitmaps  numpy_time  roaring_time  \
0      1000  0.010000            1    0.000007      0.000002   
1      1000  0.010000            2    0.000006      0.000003   
2      1000  0.010000            4    0.000006      0.000003   
3      1000  0.010000            8    0.000006      0.000003   
4      1000  0.100000            1    0.000005      0.000001   
5      1000  0.100000            2    0.000005      0.000002   
6      1000  0.100000            4    0.

In [26]:
results_df

,size,sparsity,num_bitmaps,numpy_time,roaring_time,numpy_storage_mb,roaring_storage_mb,numpy_peak_mb,roaring_peak_mb,intersection_size,time_speedup,storage_ratio,peak_mem_ratio
0,1000,0.010000,1,0.000007,0.000002,0.000954,0.000000,0.000000,0.000000,10.000000,3.829268,inf,NaN
1,1000,0.010000,2,0.000006,0.000003,0.001907,0.000000,0.000000,0.000000,0.000000,2.129032,inf,NaN
2,1000,0.010000,4,0.000006,0.000003,0.003815,0.000000,0.000000,0.000000,0.000000,2.283019,inf,NaN
3,1000,0.010000,8,0.000006,0.000003,0.007629,0.000000,0.000000,0.000000,0.000000,2.148148,inf,NaN
4,1000,0.100000,1,0.000005,0.000001,0.000954,0.000000,0.000000,0.000000,100.000000,3.354839,inf,NaN
5,1000,0.100000,2,0.000005,0.000002,0.001907,0.000000,0.000000,0.000000,11.200000,2.291667,inf,NaN
6,1000,0.100000,4,0.000005,0.000003,0.003815,0.000000,0.000000,0.000000,0.200000,2.017857,inf,NaN
7,1000,0.100000,8,0.000006,0.000003,0.007629,0.000000,0.000000,0.000000,0.000000,1.875000,inf,NaN
8,100000,0.010000,1,0.000011,0.000003,0.095367,0.000000,0.000000,0.000000,1000.000000,3.477612,inf,NaN
9,100000,0.010000,2,0.000016,0.000006,0.190735,0.000000,0.000000,0.000000,8.800000,2.664000,inf,NaN
